# add scraper to this ipynb

---

QUESTION 1: Factors that impact salary

To predict salary you will be building either a classification or regression model, using features like the location, title, and summary of the job. If framing this as a regression problem, you will be estimating the listed salary amounts. You may instead choose to frame this as a classification problem, in which case you will create labels from these salaries (high vs. low salary, for example) according to thresholds (such as median salary).

You have learned a variety of new skills and models that may be useful for this problem:

NLP
Unsupervised learning and dimensionality reduction techniques (PCA, clustering)
Ensemble methods and decision tree models
SVM models
Whatever you decide to use, the most important thing is to justify your choices and interpret your results. Communication of your process is key. Note that most listings DO NOT come with salary information. You'll need to able to extrapolate or predict the expected salaries for these listings.

QUESTION 2: Factors that distinguish job category

Using the job postings you scraped for part 1 (or potentially new job postings from a second round of scraping), identify features in the data related to job postings that can distinguish job titles from each other. There are a variety of interesting ways you can frame the target variable, for example:

What components of a job posting distinguish data scientists from other data jobs?
What features are important for distinguishing junior vs. senior positions?
Do the requirements for titles vary significantly with industry (e.g. healthcare vs. government)?
You may end up making multiple classification models to tackle different questions. Be sure to clearly explain your hypotheses and framing, any feature engineering, and what your target variables are. The type of classification model you choose is up to you. Be sure to interpret your results and evaluate your models' performance.

BONUS PROBLEM

Your boss would rather tell a client incorrectly that they would get a lower salary job than tell a client incorrectly that they would get a high salary job. Adjust one of your models to ease his mind, and explain what it is doing and any tradeoffs. Plot the ROC curve.

---

In [83]:
import numpy as np
import pandas as pd
import requests
import cPickle as pickle
from bs4 import BeautifulSoup
import time
from time import sleep

In [84]:
indeed_cities = ['New+York', 'Chicago', 'San+Francisco', 'San+Jose', 'San+Diego', 'Los+Angeles', 'Washington%2C+DC', 'Boston', 'Pittsburgh', 'Philadelphia', 'Atlanta', 'Cincinnati', 'St.+Louis', 'Tampa', 'Oakland', 'Austin', 'Houston', 'Dallas', 'Seattle', 'Portland', 'Denver', 'Phoenix', 'Minneapolis', 'Miami', 'Charlotte', 'Jacksonville', 'Indianapolis', 'Nashville', 'Kansas+City', 'Columbus']
len(indeed_cities)

30

, 'San+Jose', 'San+Diego', 'Los+Angeles', 'Washington%2C+DC',
          'Boston', 'Pittsburgh', 'Philadelphia', 'Atlanta', 'Cincinnati', 'St.+Louis', 'Tampa', 'Oakland',
          'Austin', 'Houston', 'Dallas', 'Seattle', 'Portland', 'Denver', 'Phoenix', 'Minneapolis', 'Miami',
          'Charlotte', 'Jacksonville', 'Indianapolis', 'Nashville', 'Kansas+City', 'Columbus'

---

scrape 1 location

In [90]:
title = []
company = []
location = []
salary = []
summary = []
skills = []
#copy from scraper ipynb 1 loc, no of search result.

def scrape(city, results):
    
    def get_title(soup): 
        jobs = []
        for div in soup.find_all('div', attrs={'class':'row'}):
            for a in div.find_all('a', attrs={'data-tn-element':'jobTitle'}):
                jobs.append(a['title'])
        return jobs

    def get_company(soup): 
        companies = []
        for div in soup.find_all('div', attrs={'class':'row'}):
            company = div.find_all('span', attrs={'class':'company'})
            if len(company) > 0:
                for b in company:
                    companies.append(b.text.strip())
            else:
                sec_try = div.find_all('span', attrs={'class':'result-link-source'})
                for span in sec_try:
                    companies.append(span.text.strip())

        return companies

    def get_location(soup): 
        locations = []
        spans = soup.find_all('span', attrs={'class': 'location'})
        for span in spans:
            locations.append(span.text)
        return locations

    def get_salary(soup): 
        salaries = []
        for div in soup.find_all('div', attrs={'class':'row'}):
            try:
                salaries.append(div.find('nobr').text)
            except:
                try:
                    div_two = div.find(name='div',attrs={'class':'sjcl'})
                    div_three = div_two.find('div')
                    salaries.append(div_three.text.strip())
                except:
                    salaries.append('NA')
        return salaries    
    
    def get_summary(soup): 
        summaries = []
        spans = soup.find_all('span', attrs={'class': 'summary'})
        for span in spans:
            summaries.append(span.text.strip())
        return summaries
    
    def get_skill(soup): 
        skills = []
        for div in soup.find_all('div', attrs={'class':'row'}):
            try:
                skills.append(div.find('span', attrs={'class': 'experienceList'}).text)
            except:
                skills.append('NA')
        return skills  
    
    
  #for city in indeed_cities:
   # for start in range(0, max_results_per_city, 100):
    #    url = "https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=\
     #          &salary=&radius=25&l=" + city + "&fromage=any&limit=100&start=" + str(start) + "&sort=&psf=advsrch"
      #  html = requests.get(url)
       # soup = BeautifulSoup(html.text, 'html.parser')
        #for result in soup.find_all('div', {'class':' row result'}):
         #   results.append(result)
       # sleep(1)
        
    for start in range (0, results, 50):
        url = 'https://www.indeed.com/jobs?q=' + 'data+scientist' + '&l=' + city \
            + '&limit=50&radius=25' + '&start=' + str(start)
        request = requests.get(url)
        soup = BeautifulSoup(request.text, 'html.parser')
    
        title.extend(get_title(soup))
        company.extend(get_company(soup))
        location.extend(get_location(soup))
        salary.extend(get_salary(soup))
        summary.extend(get_summary(soup))
        skills.extend(get_skill(soup))
        time.sleep(3)
        print start, 'scraped.'

In [91]:
max_results = 1000

for city in indeed_cities:
    scrape(city,max_results)
    sleep(30)

0 scraped.
50 scraped.
100 scraped.
150 scraped.
200 scraped.
250 scraped.
300 scraped.
350 scraped.
400 scraped.
450 scraped.


KeyboardInterrupt: 

In [5]:
len(title)

#title

6132

In [6]:
len(salary)

6132

In [7]:
len(location)


6132

In [8]:
len(company)

6132

In [12]:
df = pd.DataFrame(columns = ['title','company','location','salary','summary','skill'])
df.title = title
df.company = company
df.location = location
df.salary = salary
df.summary = summary
df.skill = skills

df.shape

(6132, 6)

In [15]:
df.head()

df.isnull().sum()

title       0
company     0
location    0
salary      0
summary     0
skill       0
dtype: int64

In [18]:
df.to_pickle("./indeed.pkl")

In [57]:
new_df = pd.read_pickle("./indeed.pkl")

file1 = open("./indeed.pkl",'r')
openfile = pickle.load(file1)
openfile

,title,company,location,salary,summary,skill
0,"Associate, Data Scientist",Morgan Stanley,"New York, NY 10032 (Washington Heights area)",NA,Utilize experience in applying Machine Learnin...,NA
1,"Data Scientist/Quantitative Analyst, Engineering",Google,"New York, NY 10011 (Chelsea area)",NA,"4 years of relevant work experience (e.g., as ...",NA
2,Data Scientist,Capital One,"New York, NY",NA,Integrating with external data sources and API...,NA
3,Data Scientist,Bank of America,"New York, NY 10020 (Midtown area)",NA,As well as deployment of advanced techniques (...,NA
4,Data Science Analyst,Aetna,"New York, NY 10016 (Gramercy area)",NA,This position will use predictive and behavior...,NA
5,"Data Scientist, Analytics",Facebook,"New York, NY",NA,The Data Scientist Analytics role has work acr...,NA
6,Data Scientist - Worldwide Advanced Analytics ...,IBM,"New York, NY",NA,As a member of our team you will get to work h...,NA
7,Research Scientist - All Levels (Applied Data ...,Columbia University,"New York, NY",NA,Contribute to research projects involving one ...,NA
8,"Quantitative Analyst Intern, Summer 2018",Google,"New York, NY 10011 (Chelsea area)",NA,"Specifically, we support the development of in...",NA
9,Technology - GSAM Tech - Data Science and Mach...,Goldman Sachs,"New York, NY 10282 (Tribeca area)",NA,IMD Knowledge Management team is looking to ex...,NA


In [58]:
# Check our rows after removing duplicates

df1 = new_df[~new_df.duplicated()]

df1.shape

(5012, 6)

In [66]:
df1.head()

,title,company,location,salary,summary,skill
0,"Associate, Data Scientist",Morgan Stanley,"New York, NY 10032 (Washington Heights area)",NA,Utilize experience in applying Machine Learnin...,NA
1,"Data Scientist/Quantitative Analyst, Engineering",Google,"New York, NY 10011 (Chelsea area)",NA,"4 years of relevant work experience (e.g., as ...",NA
2,Data Scientist,Capital One,"New York, NY",NA,Integrating with external data sources and API...,NA
3,Data Scientist,Bank of America,"New York, NY 10020 (Midtown area)",NA,As well as deployment of advanced techniques (...,NA
4,Data Science Analyst,Aetna,"New York, NY 10016 (Gramercy area)",NA,This position will use predictive and behavior...,NA


---

---

In [65]:
#print df1.dtypes

df1.location = df1.location.astype(str)

print df1.dtypes

title       object
company     object
location    object
salary      object
summary     object
skill       object
dtype: object


In [34]:

def to_state(loc):
    if 'CA' in loc:
        loc = 'California'
    elif 'AZ' in loc:
        loc = 'Arizona'
    elif 'CO' in loc:
        loc = 'Colorado'
    elif 'CT' in loc:
        loc = 'Connecticut'
    elif 'MA' in loc:
        loc = 'Massachusetts'
    elif 'NY' in loc:
        loc = 'New York'
    elif 'NC' in loc:
        loc = 'North Carolina'
    elif 'TX' in loc:
        loc = 'Texas'
    elif 'PA' in loc:
        loc = 'Pennsylvania'
    elif 'VA' in loc:
        loc = 'Virginia'
    elif 'WA' in loc:
        loc = 'Washington'
    elif 'FL' in loc:
        loc = 'Florida'
    elif 'IL' in loc:
        loc = 'Illinois'
    elif 'IN' in loc:
        loc = 'Indiana'
    elif 'IA' in loc:
        loc = 'Iowa'
    elif 'MD' in loc:
        loc = 'Maryland'
    elif 'New York' in loc:
        loc = 'New York'
    else:
        loc = loc
    return loc

In [67]:
# Apply function to change values to state names
df1.location = df1.location.map(to_state)

In [68]:
# Check values of location column to make sure there are only state names, and no stray values
df1.location.value_counts()

California                                                1777
Pennsylvania                                               606
Illinois                                                   482
New York                                                   461
Massachusetts                                              414
St. Louis, MO                                              201
Virginia                                                   199
Atlanta, GA                                                194
Washington, DC                                             113
Maryland                                                    86
Cincinnati, OH                                              77
Alpharetta, GA                                              42
Wilmington, DE                                              24
Atlanta, GA 30318 (Rockdale area)                           19
Atlanta, GA 30338                                           14
Wilmington, DE 19801                                   

In [76]:
# Remove and replace certain symbols for easier processing thereafter (splitting by '-' to obtain each element)
df1.salary = df1.salary.map(lambda x: x.replace('\n','').replace('$','').replace(',', '').replace(' a ', '-').replace(' - ','-').replace(' an ', '-'))

In [78]:
df1['low'] = df1.salary.map(lambda x: x.split('-')[0])

/Users/Naren/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
df1['time'] = df1.salary.map(lambda x: x.split('-')[-1])

In [82]:
# Create function to get upper bound of salary

def upper(sal):
   try:
        x = sal.split('-')[1]  
        for i in x:  
            if i in 'abcdefghijklmnopqrstuvwxyz':  
                x = np.nan 
                return x
                except:
                    return np.nan

SyntaxError: invalid syntax (<ipython-input-82-fe62184fe49c>, line 10)

In [41]:
# Map function to create column for upper bound of salary
#df['r2'] = df.salary.map(r2)

In [42]:
#df.head()

,title,company,location,salary,summary,skill,r1,per,r2
0,"Associate, Data Scientist",Morgan Stanley,New York,NA,Utilize experience in applying Machine Learnin...,NA,NA,NA,NaN
1,"Data Scientist/Quantitative Analyst, Engineering",Google,New York,NA,"4 years of relevant work experience (e.g., as ...",NA,NA,NA,NaN
2,Data Scientist,Capital One,New York,NA,Integrating with external data sources and API...,NA,NA,NA,NaN
3,Data Scientist,Bank of America,New York,NA,As well as deployment of advanced techniques (...,NA,NA,NA,NaN
4,Data Science Analyst,Aetna,New York,NA,This position will use predictive and behavior...,NA,NA,NA,NaN


IndexError: list index out of range